In [95]:

# coding: utf-8
# <nbformat>3.0</nbformat>

# <markdowncell>
# <codecell>


# In[1]:

import pandas as pd
import numpy as np
import os 
from collections import Counter
from scipy import spatial
import sys
import networkx as nx
sys.setrecursionlimit(10000)


def negative_eQTL_oneSNP(eQTL, distance_threshold):
    '''
    match the eQTL using chr and distance
    '''
    eSNP_pos = eQTL['SNP_POS']
    echr = eQTL['SNP_CHR']
    eDistance = eQTL['DISTANCE']
    egene = eQTL['GENE']
    
    genes_chr = GRCh37_genes[GRCh37_genes['Chromosome/scaffold name'] == str(echr)]
    t = genes_chr.apply(lambda x: abs(x['Gene Start (bp)'] - eSNP_pos) < distance_threshold, axis=1)
    neg_gene = genes_chr[t].sample(1)
    
    if sum(t)>1:
        while neg_gene.index[0] == egene:
            neg_gene = genes_chr[t].sample(1)
    else: 
        print "No negative gene found"
        
    return neg_gene.index[0]
    


# In[3]:

def negative_eQTL_onecell(eQTL, distance_threshold = 1000000):
    
    eQTL_nega = pd.DataFrame()
    
    for name, df in eQTL.groupby('SNP_CHR'):
        eQTL_pergene = df.loc[df.groupby('GENE')['P-VALUE'].idxmin()]    ## One Lead SNP per gene
        eQTL_pergene = eQTL_pergene.reset_index(drop=True)
        negative_genes = eQTL_pergene.apply(lambda x: negative_eQTL_oneSNP(x,distance_threshold=distance_threshold),axis=1)
        negative_genes = GRCh37_genes.loc[np.array(negative_genes)].reset_index(drop=True)

        eQTL_nega_pergene = eQTL_pergene.merge(negative_genes, left_index=True, right_index=True)
        eQTL_nega_pergene = eQTL_nega_pergene[['SNP', 'SNP_CHR', 'SNP_POS', 'Associated Gene Name', 'Gene Start (bp)','P-VALUE']]
        eQTL_nega_pergene.columns = [['SNP', 'SNP_CHR', 'SNP_POS', 'GENE', 'GENE_START_POS','P-VALUE']]
        eQTL_nega_pergene['DISTANCE'] = abs(eQTL_nega_pergene['SNP_POS'] - eQTL_nega_pergene['GENE_START_POS'])
        
        eQTL_nega = eQTL_nega.append(eQTL_nega_pergene)
        
    return eQTL_nega


# In[4]:

def eQTL_in_fragments_one_chr(eQTL_df, contacting_df, SNP_window, promoter_window, 
                              fragments = ['baitStart', 'baitEnd', 'oeStart', 'oeEnd']):
    '''
    e-SNP near bait/oe, and promoter of the e-gene near oe/bait
    '''
    
    N = len(contacting_df)
    Start1, End1, Start2, End2 = fragments

    # e-SNP
    SNP_positions = np.reshape(eQTL_df['SNP_POS'],[len(eQTL_df),1])
    SNP_tree = spatial.KDTree(SNP_positions)

    frag1_start = np.reshape(contacting_df[Start1],[N,1])
    SNP_near_frag1Start = SNP_tree.query_ball_point(frag1_start, SNP_window)
    frag1_end = np.reshape(contacting_df[End1],[N,1])
    SNP_near_frag1End = SNP_tree.query_ball_point(frag1_end, SNP_window)

    SNP_near_frag1 = [SNP_near_frag1Start[i] + SNP_near_frag1End[i] for i in xrange(N)]


    # e-gene
    gene_start = np.reshape(eQTL_df['GENE_START_POS'], [len(eQTL_df),1])
    gene_tree = spatial.KDTree(gene_start)

    frag2_start = np.reshape(contacting_df[Start2], [N,1])
    gene_near_frag2Start = gene_tree.query_ball_point(frag2_start, promoter_window)
    frag2_end = np.reshape(contacting_df[End2], [N,1])
    gene_near_frag2End = gene_tree.query_ball_point(frag2_end, promoter_window)

    gene_near_frag2 = [gene_near_frag2Start[i] + gene_near_frag2End[i] for i in xrange(N)]
    
    SNPID = np.array(SNP_near_frag1)[np.array([len(np.intersect1d(SNP_near_frag1[t],gene_near_frag2[t]))>0 
                                               for t in xrange(len(SNP_near_frag1))])]
    geneID = np.array(gene_near_frag2)[np.array([len(np.intersect1d(SNP_near_frag1[t],gene_near_frag2[t]))>0 
                                                 for t in xrange(len(SNP_near_frag1))])]
    QTLID = [list(np.intersect1d(SNPID[t], geneID[t])) for t in xrange(len(SNPID))]
    
    return list(set([x for sublist in QTLID for x in sublist]))






# In[5]:

def eQTL_in_fragments_one_cell(eQTL, pairs, cell, SNP_window, promoter_window):
    
    contacting_cell = pairs[list(pairs.columns[:11])+[cell]]  
    contacting_cell = contacting_cell[contacting_cell[cell] > 0]   

    result_df = pd.DataFrame()
    for name, df in eQTL.groupby('SNP_CHR'):
        contacting_pairs = contacting_cell[contacting_cell['baitChr'] == str(name)]
        eQTL_pergene = df.loc[df.groupby('GENE')['P-VALUE'].idxmin()]    ## One Lead SNP per gene
        
        functional_contacting_idx1 = eQTL_in_fragments_one_chr(eQTL_pergene, contacting_pairs, SNP_window, promoter_window)
        functional_contacting_idx2 = eQTL_in_fragments_one_chr(eQTL_pergene, contacting_pairs, SNP_window, promoter_window,
                                                               fragments = ['oeStart', 'oeEnd','baitStart', 'baitEnd'])
        contact_vector = np.array([0] * len(eQTL_pergene))
        contact_vector[np.array(list(set(functional_contacting_idx1+functional_contacting_idx2)))] = 1
        eQTL_pergene['contacting'] = contact_vector
        
        result_df = result_df.append(eQTL_pergene)
        
        print 'There are %i contacting eQTL pairs among %i for chr%i' % (sum(contact_vector),len(eQTL_pergene), name)
                
    return result_df



# In[6]:

def save_bed_files(result_df, celltype, SNP_window, promoter_window, negative_set = False):
    '''
    save the bed files for intersection with TFB motif
    '''
    
    SNP_bed = []
    SNP_bed.append(['chr%i'% x for x in (result_df['SNP_CHR'])])
    SNP_bed.append(list(np.array(result_df['SNP_POS']) - SNP_window))
    SNP_bed.append(list(np.array(result_df['SNP_POS']) + SNP_window))
    SNP_bed.append(list(result_df['SNP']))
    SNP_bed = pd.DataFrame(SNP_bed).transpose()
    if negative_set:
        promoter_bed.to_csv(os.path.join(DATA_DIR, 'intermediate/%s/PCHiC_peak_matrix_cutoff5_%s_eSNP_negativeset.bed'% (celltype, celltype)),
                   sep='\t',index=False, header=False)
    else:
        SNP_bed.to_csv(os.path.join(DATA_DIR, 'intermediate/%s/PCHiC_peak_matrix_cutoff5_%s_eSNP.bed' % (celltype, celltype)),
                   sep='\t',index=False, header=False)
    
    ## 
    promoter_bed = []
    promoter_bed.append(['chr%i'% x for x in (result_df['SNP_CHR'])])
    promoter_bed.append(list(np.array(result_df['GENE_START_POS']) - promoter_window))
    promoter_bed.append(list(np.array(result_df['GENE_START_POS']) + promoter_window))
    promoter_bed.append(list(result_df['GENE']))
    promoter_bed = pd.DataFrame(promoter_bed).transpose()
    if negative_set:
        promoter_bed.to_csv(os.path.join(DATA_DIR, 'intermediate/%s/PCHiC_peak_matrix_cutoff5_%s_egene_negativeset.bed'% (celltype, celltype)),
                   sep='\t',index=False, header=False)
    else:
        promoter_bed.to_csv(os.path.join(DATA_DIR, 'intermediate/%s/PCHiC_peak_matrix_cutoff5_%s_egene.bed'% (celltype, celltype)),
                   sep='\t',index=False, header=False)
        
        
def readin_intermediate_result(celltype, SNP_window, promoter_window, negative_set=False):
    if negative_set:
        fn_SNP = 'PCHiC_peak_matrix_cutoff5_%s_eSNP_negative.bed' % chr
        fn_gene = 'PCHiC_peak_matrix_cutoff5_%s_egene_negative.bed' % chr
    else:
        fn_SNP = 'PCHiC_peak_matrix_cutoff5_%s_eSNP.bed' % chr
        fn_gene = 'PCHiC_peak_matrix_cutoff5_%s_egene.bed' % chr
    a = pd.read_csv(os.path.join(DATA_DIR, 'intermediate/%s/%s' % (celltype,fn_SNP)),sep='\t',header=None)
    b = pd.read_csv(os.path.join(DATA_DIR, 'intermediate/%s/%s' % (celltype,fn_gene)),sep='\t',header=None)
    
    inter_result = pd.concat((a,b),axis=1)
    inter_result.columns = ['SNP_CHR','SNP_POS','non','SNP','GENE_chr','GENE_START_POS','NAN','GENE']
    inter_result = inter_result[['SNP_CHR','SNP_POS','SNP','GENE_chr','GENE_START_POS','GENE']]
    inter_result['SNP_POS'] = inter_result['SNP_POS'] + SNP_window
    inter_result['GENE_START_POS'] = inter_result['GENE_START_POS'] + promoter_window
    return inter_result

# In[ ]:

#### Need to scan the bed files for known motifs using fimo
#### run the following codes in DATA_DIR
#### bash TFBS.sh Mon eSNP
#### bash TFBS.sh Mon gene
#### bash TFBS.sh tCD4 eSNP
#### bash TFBS.sh tCD4 gene


# In[ ]:

-

# In[78]:

def eQTL_in_ATACseq_one_chr(eQTL_pos, ATAC_regions, window = 1000):

    N = len(ATAC_regions)
    eQTL_tree = spatial.KDTree(eQTL_pos)

    frag_start = np.reshape(ATAC_regions['Start'],[N,1])
    eQTL_near_fragStart = eQTL_tree.query_ball_point(frag_start, window)
    frag_end = np.reshape(ATAC_regions['End'],[N,1])
    eQTL_near_fragEnd = eQTL_tree.query_ball_point(frag_end, window)

    eQTL_near_frag = [list(set(eQTL_near_fragStart[i] + eQTL_near_fragEnd[i])) for i in xrange(N)]
    eQTLID = [x for x in eQTL_near_frag if len(x) > 0]
    eQTLID = list(set([a for b in eQTLID for a in b]))
    
    x = np.zeros(len(eQTL_pos))
    x[eQTLID] = 1
    return x


# In[119]:

def eQTL_in_ATACseq_one_cell(eQTL, ATAC, window_SNP = 1000, window_gene = 2000000):
    
    result_df = pd.DataFrame()
    for name, g in eQTL.groupby('SNP_CHR'):
        ## eSNP
        eSNP_pos = np.reshape(g['SNP_POS'],[len(g),1])
        ATAC_SNP_vector = eQTL_in_ATACseq_one_chr(eSNP_pos,ATAC[ATAC['Chr'] == name], window = window_SNP)
        ## egene
        egene_pos = np.reshape(g['GENE_START_POS'],[len(g),1])
        ATAC_gene_vector = eQTL_in_ATACseq_one_chr(egene_pos,ATAC[ATAC['Chr'] == name], window = window_gene)
        
        ## ATAC profile
        ATAC_eQTL = pd.DataFrame({'ATAC_SNP':ATAC_SNP_vector,'ATAC_gene':ATAC_gene_vector})
        g = pd.concat((g.reset_index(drop=True),ATAC_eQTL),axis=1)
        result_df = result_df.append(g)        
        
    return result_df


In [48]:
def SNP_in_PIR(eQTL, contacting_df, PIR_window = 500):
    gene_promoter = eQTL['GENE']
    SNP = eQTL['SNP']
    SNP_POS = eQTL['SNP_POS']

    gene_promoter_idx = np.where([gene_promoter in x.split(';') for x in list(contacting_df['baitName'])])[0]
    gene_promoter_window = contacting_df.iloc[gene_promoter_idx]

    left = np.array(gene_promoter_window['oeStart']) - PIR_window
    right = np.array(gene_promoter_window['oeEnd']) + PIR_window

    if sum([(t[0] < SNP_POS and t[1] > SNP_POS) for t in zip(left,right)]) > 0:
        return 1
    else:
        return 0
    
    
    
def eQTL_in_fragments_one_cell(eQTL, pairs, cell, SNP_window, promoter_window):
    
    contacting_cell = pairs[list(pairs.columns[:11])+[cell]]  
    contacting_cell = contacting_cell[contacting_cell[cell] > 0]   

    result_df = pd.DataFrame()
    for name, df in eQTL.groupby('SNP_CHR'):
        contacting_pairs = contacting_cell[contacting_cell['baitChr'] == str(name)]
        eQTL_pergene = df.loc[df.groupby('GENE')['P-VALUE'].idxmin()]    ## One Lead SNP per gene
        
        functional_contacting_idx = eQTL_pergene.apply(lambda x: SNP_in_PIR(x, contacting_pairs),axis=1)
        contact_vector = np.array(functional_contacting_idx)
        eQTL_pergene['contacting'] = contact_vector
        
        result_df = result_df.append(eQTL_pergene)
        
        print 'There are %i contacting eQTL pairs among %i for chr%i' % (sum(contact_vector),len(eQTL_pergene), name)
                
    return result_df






In [ ]:
DATA_DIR = '/Users/Yuan/Documents/BLab/Predict_target_genes/data'

compute_negative_set = True
match_distance = 1000000

from_beginning=True
negative_set_flag = True

SNP_window_for_PHiC = 1000
promoter_window_for_PHiC = 1000000

SNP_window_for_motif = 1000
promoter_window_for_motif = 2000

if compute_negative_set:
    ### read in all genes infomation
    GRCh37_genes = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/GRCh37_genes.txt'),sep='\t')
    gene_idx = []
    for name, df in GRCh37_genes.groupby(['Associated Gene Name']):
        gene_idx.append(np.random.choice(list(df.index)))
    GRCh37_genes = GRCh37_genes.iloc[np.array(gene_idx)]
    GRCh37_genes.index = GRCh37_genes['Associated Gene Name']
    
    ### compute negative sets match for chr and distance
    CD4_eQTL = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/Raj/cd4T_cis_fdr05.tsv'),sep='\t')
    mon_eQTL = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/Raj/monocytes_cis_fdr05.tsv'),sep='\t')
    CD4_eQTL_neg = negative_eQTL_onecell(CD4_eQTL, match_distance)
    CD4_eQTL_neg.to_csv(os.path.join(DATA_DIR, 'eQTL/Raj/cd4T_cis_fdr05_negative.tsv'),sep='\t',index=False)
    mon_eQTL_neg = negative_eQTL_onecell(mon_eQTL, match_distance)
    mon_eQTL_neg.to_csv(os.path.join(DATA_DIR, 'eQTL/Raj/monocytes_cis_fdr05_negative.tsv'),sep='\t',index=False)

    
if from_beginning:
    ### First use all contacting pairs (could transfer to contacting E_P pairs later)
    pairs = pd.read_csv(os.path.join(DATA_DIR,'CPHiC/Interactions/PCHiC_peak_matrix_cutoff5.txt'),sep='\t', low_memory=False)
    pairs = pairs[pairs['baitChr'] == pairs['oeChr']]
    
    ### read in the (negative) eQTL list
    if negative_set_flag:
        CD4_eQTL = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/Raj/cd4T_cis_fdr05_negative.tsv'),sep='\t')
        mon_eQTL = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/Raj/monocytes_cis_fdr05_negative.tsv'),sep='\t')
    else:
        CD4_eQTL = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/Raj/cd4T_cis_fdr05.tsv'),sep='\t')
        mon_eQTL = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/Raj/monocytes_cis_fdr05.tsv'),sep='\t')
        
    ### compute the overlap of baits/oe with the eQTL list
    print "tCD4"
    tCD4_result = eQTL_in_fragments_one_cell(CD4_eQTL, pairs, 'tCD4',SNP_window_for_PHiC, promoter_window_for_PHiC)
    save_bed_files(tCD4_result, 'tCD4', SNP_window_for_motif, promoter_window_for_motif, negative_set_flag)
    print "Mon"
    Mon_result = eQTL_in_fragments_one_cell(mon_eQTL, pairs, 'Mon',SNP_window_for_PHiC, promoter_window_for_PHiC)
    save_bed_files(Mon_result, 'Mon', SNP_window_for_motif, promoter_window_for_motif, negative_set_flag)
    
else:
    ### read directly from the intermediate results
    if negative_set_flag:
        Mon_result = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/FairFax/%s_cis_%i_fdr05_50kb_negative_annotated.csv' % (celltype,chr)), sep='\t')
    else:
        Mon_result = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/FairFax/%s_cis_%i_fdr05_50kb_annotated.csv' % (celltype,chr)), sep='\t')
        
    ### add on the TF motif information
    Mon_result = merge_TF_motif(Mon_result, 'Mon')



In [99]:

# In[120]:

#### ATAC-seq

ATACseq = pd.read_csv(os.path.join(DATA_DIR, 'ATACseq/GSE74912_ATACseq_All_Counts.txt'),sep='\t')
ATAC_CD4 = pd.concat((ATACseq[['Chr','Start','End']],
#                      ATACseq[[x for x in ATACseq.columns if 'CD4' in x]].mean(axis=1)),axis=1)
tCD4_TF_ATAC = eQTL_in_ATACseq_one_cell(tCD4_TF, ATACseq)




,SNP,SNP_CHR,SNP_POS,GENE,GENE_START_POS,P-VALUE,DISTANCE,SNP_contacting_df_idx,contacting,motif_SNP,motif_gene
0,rs6701264,1,34272479,A3GALT2,33545037,2.120000e-04,727442,[],0,[MA0050.2],"[MA0014.2, MA0050.2]"
1,rs11804707,1,10689053,AADACL4,12649347,1.602000e-03,1960294,[],0,[MA0050.2],[MA0050.2]
2,rs7518038,1,75927018,ACADM,75999445,4.940000e-07,72427,[],0,[MA0050.2],[MA0050.2]
3,rs2986739,1,6470643,ACOT7,6246931,1.260000e-05,223712,"[818, 841, 854, 971, 1014]",2,[MA0050.2],"[MA0014.2, MA0139.1, MA0050.2]"
4,rs12040970,1,153361924,ADAM15,153299393,5.120000e-09,62531,[36135],0,[MA0050.2],[MA0050.2]


In [3]:
def map_to_1based(anarray):
    return dict(zip(set(anarray), xrange(len(anarray))))


def bait_names(mapped_1based_idx, names):
    return dict(zip(mapped_1based_idx, names))

def edges_inbetween(D, n_nodes):
    ### input: a graph object
    ### return the degree of contacts between nodes: 1/2/0
    goal_matrix = np.ones([n_nodes,n_nodes]) * 100
    for n,nbrs in D.adjacency_iter():
        # assign 1 to the direct linkage
        goal_matrix[n][D[n].keys()] = 1
        # assign 2 to the indirect linkage
        for t in D[n].keys():
            goal_matrix[n][D[t].keys()] = map(lambda x: min(x), 
                                              zip(list(goal_matrix[n][D[t].keys()]),
                                                  [goal_matrix[n][t]+1] * len(goal_matrix[n][D[t].keys()])))
    goal_matrix[goal_matrix==100] = 0
    np.fill_diagonal(goal_matrix,0)
    return goal_matrix



def contacting_to_degree(data,edge_threshold=0):
    ## obtain the nodes and the weights edges
    if edge_threshold != 0:
        data = data.iloc[np.where(data['obs_exp']>edge_threshold)[0]]
    fragment_len = max(max(data['i']),max(data['j'])) + 1
    nodes = range(fragment_len)
    weighted_edges = zip(list(data['i']),list(data['j']),list(data['obs_exp']))
    print fragment_len
    
    ## construct the graph
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_weighted_edges_from(weighted_edges)
    
    ## obtain number of edges between the bins
    number_of_edges_inbetween = edges_inbetween(G, fragment_len)
    return G, number_of_edges_inbetween

In [4]:
def compute_frag_by_frag(d, celltype):
    anarray = list(d['baitID']) + list(d['oeID'])
    t = map_to_1based(anarray)
    d['baitID'] = map(lambda x: t[x], list(d['baitID']))
    d['oeID'] = map(lambda x: t[x], list(d['oeID']))
    baitsnames = bait_names(list(d['baitID']) + list(d['oeID']), list(d['baitName']) + list(d['oeName']))

    df = d[['baitID','oeID',celltype]]
    df.columns = ['i','j','obs_exp']

    df = df.reset_index(drop=True)

    G,frag_by_frag = contacting_to_degree(df,edge_threshold=0)
    frag_by_frag = pd.DataFrame(frag_by_frag)
    frag_by_frag.index = baitsnames.values()
    frag_by_frag.loc['NOGENE'] = [0]*len(frag_by_frag)
    
    frag_by_frag = frag_by_frag.drop(["."])
    frag_by_frag = frag_by_frag.to_dict(orient = 'index')
    
    return frag_by_frag

In [32]:
def find_the_key(frag_by_frag, gg):
    gg_in_frag = np.where(map(lambda x: gg in x, frag_by_frag.keys()))[0]
    if len(gg_in_frag) > 0:
        temp = np.array(frag_by_frag.keys())[np.array(gg_in_frag)]
        return np.random.choice(temp)
    else:
        return 'NOGENE'
    

def degree_of_contact_SNP_gene(eQTL_df, contacting_df, frag_by_frag):

    ### find the other fragments where eSNP locate in 
    left = np.array(contacting_df['oeStart']) - 500
    right = np.array(contacting_df['oeEnd']) + 500
    SNPs_PIR = eQTL_df.apply(lambda x: [x['SNP_POS'] in range(t[0],t[1]) for t in zip(left,right)], axis=1)
    SNPs_PIR = SNPs_PIR.apply(lambda x: list(np.where(x)[0]))   ## a list of lists
    eQTL_df['SNP_contacting_df_idx'] = list(SNPs_PIR)
    
    ### find the degree of contact by using frag_by_frag matrix
    degree_contact = eQTL_df.apply(lambda eq: [frag_by_frag[find_the_key(frag_by_frag, eq['GENE'])][contacting_df.iloc[t]['oeID']] 
                                               for t in eq['SNP_contacting_df_idx']], axis=1)
    
    return degree_contact

In [ ]:
DATA_DIR = '/Users/Yuan/Documents/BLab/Predict_target_genes/data'

if 1:
    pairs = pd.read_csv(os.path.join(DATA_DIR,'CPHiC/Interactions/PCHiC_peak_matrix_cutoff5.txt'),sep='\t', low_memory=False)
    pairs = pairs[pairs['baitChr'] == pairs['oeChr']]
    
    CD4_eQTL = pd.read_csv(os.path.join(DATA_DIR, 'eQTL/Raj/cd4T_cis_fdr05_negative.tsv'),sep='\t')



In [17]:
contacting_df = pairs[pairs['baitChr'] == '22']
eQTL_df = CD4_eQTL[CD4_eQTL['SNP_CHR'] == 22]
eQTL_df = eQTL_df.iloc[10:20]

In [34]:
frag2 = compute_frag_by_frag(contacting_df,'tCD4') 
degree_contact = degree_of_contact_SNP_gene(eQTL_df, contacting_df, frag2)

contact_vector = np.zeros(len(eQTL_df))
contact_vector[np.where(map(lambda x: 1 in x, degree_contact))[0]] = 1
contact_vector[np.where(map(lambda x: 2 in x, degree_contact))[0]] = 2



2904


/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [47]:
if 1:
    eQTL = eQTL_df
    cell = 'tCD4'
    
    print 'Compute whether the eQTL pair fall in any promoter-PIR pair'
    
    contacting_cell = pairs[list(pairs.columns[:11])+[cell]]  
    contacting_cell = contacting_cell[contacting_cell[cell] > 0]   

    result_df = pd.DataFrame()
    for name, df in eQTL.groupby('SNP_CHR'):
        contacting_pairs = contacting_cell[contacting_cell['baitChr'] == str(name)]
        eQTL_pergene = df.loc[df.groupby('GENE')['P-VALUE'].idxmin()]    ## One Lead SNP per gene
        
        # functional_contacting_idx1 = eQTL_in_fragments_one_chr(eQTL_pergene, contacting_pairs, SNP_window, promoter_window)
        # functional_contacting_idx2 = eQTL_in_fragments_one_chr(eQTL_pergene, contacting_pairs, SNP_window, promoter_window,
        #                                                        fragments = ['oeStart', 'oeEnd','baitStart', 'baitEnd'])
        # contact_vector = np.array([0] * len(eQTL_pergene))
        # contact_vector[np.array(list(set(functional_contacting_idx1+functional_contacting_idx2)))] = 1

        frag2 = compute_frag_by_frag(contacting_pairs,cell) 
        degree_contact = degree_of_contact_SNP_gene(eQTL_pergene, contacting_pairs, frag2)

        contact_vector = np.zeros(len(eQTL_pergene))
        contact_vector[np.where(map(lambda x: 1 in x, degree_contact))[0]] = 1
        contact_vector[np.where(map(lambda x: 2 in x, degree_contact))[0]] = 2

        eQTL_pergene['contacting'] = contact_vector
        
        result_df = result_df.append(eQTL_pergene)
        print name
        print Counter(contact_vector)
  

Compute whether the eQTL pair fall in any promoter-PIR pair
2817
22
Counter({0.0: 9, 1.0: 1})


/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [112]:
temp = [x for sublist in [str(x).split(';') for x in pairs['baitName']] for x in sublist]
print t in temp

temp = [x for sublist in [str(x).split(';') for x in pairs['oeName']] for x in sublist]
print t in temp

False
False


In [115]:
temp = [x for sublist in [str(x).split(';') for x in pairs['baitName']] for x in sublist]
len(list(set(temp)))

26643